In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

/home/emreunel/anaconda3/envs/ass3/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence
/home/emreunel/anaconda3/envs/ass3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:16: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, namedtuple, defaultdict, Sequence
/home/emreunel/anaconda3/envs/ass3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

In [3]:
data = pd.read_csv("dataset/preprocessed.csv")

In [4]:
data = data.drop(data[data.target == -1].index)
data.shape

(762094, 44)

In [5]:
# Separate input features and target
targets = data.target
targets.head()

0    3
1    5
2    2
3    2
4    1
Name: target, dtype: int64

In [6]:
targets -= 1
targets.head()

0    2
1    4
2    1
3    1
4    0
Name: target, dtype: int64

In [7]:
targets = targets.to_numpy()

In [8]:
features = data.drop('target', axis=1)

In [9]:
features = features.to_numpy()

In [10]:
X_trainAndVal, X_test, y_trainAndVal, y_test = train_test_split(features, targets, test_size = 0.2, random_state = 0)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_trainAndVal, y_trainAndVal, test_size = 0.2, random_state = 0)

In [12]:
X_train.shape



(487740, 43)

In [13]:
unique_elements, counts_elements = np.unique(y_train, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[     0      1      2      3      4]
 [ 50166  55949  87609 117459 176557]]


In [14]:
sm = SMOTETomek(random_state = 27, n_jobs = -1)

X_train, y_train = sm.fit_sample(X_train, y_train)

/home/emreunel/anaconda3/envs/ass3/lib/python3.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'k' is deprecated from 0.2 and will be removed in 0.4. Use 'k_neighbors' instead.
  category=DeprecationWarning)
/home/emreunel/anaconda3/envs/ass3/lib/python3.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'm' is deprecated from 0.2 and will be removed in 0.4. Use 'm_neighbors' instead.
  category=DeprecationWarning)


MemoryError: 

In [ ]:
unique_elements, counts_elements = np.unique(y_train, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

In [ ]:
X_train.shape

In [ ]:
first_feature_count =           22
last_feature_count =            21


In [ ]:
print((X_train ).shape)
print(( y_train).shape)


In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = torch.from_numpy(class_weights)
class_weights



In [ ]:
batch_size = 100
epochs = 250
input_dim = first_feature_count + last_feature_count
output_dim = 5
lr_rate = 0.1

In [ ]:
class datasetLoad(Dataset):
    def __init__(self, features,labels):
        self.features = features
        self.labels = labels
    def __len__(self):
        return len(self.features)
    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [ ]:
X_train = X_train[:10000]
y_train = y_train[:10000]

X_val = X_val[:1000]
y_val = y_val[:1000]

X_test = X_test[:1000]
y_test = y_test[:1000]


In [ ]:
X_train = datasetLoad(X_train, y_train)
X_val = datasetLoad(X_val, y_val)
X_test = datasetLoad(X_test, y_test)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = X_train, batch_size = batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset = X_val, batch_size = batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = X_test, batch_size = batch_size, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class neuralNetwork(torch.nn.Module):
    def __init__(self, input_dim, hidden1_dim, hidden2_dim, output_dim):
        super(neuralNetwork, self).__init__()
        
        self.hidden1 = nn.Linear(input_dim, hidden1_dim)
        self.bnhidden1 = nn.BatchNorm1d(hidden1_dim)
        
        # self.hidden2 = nn.Linear(hidden1_dim, hidden2_dim)
        # self.bnhidden2 = nn.BatchNorm1d(hidden2_dim)
        
        self.output = nn.Linear(hidden1_dim, output_dim)

    def forward(self, x):
        
        x = self.hidden1(x)
#         x = F.relu(x)
#         x = F.elu_(x, alpha=1.) 
        x = F.leaky_relu_(self.bnhidden1(x), negative_slope=0.01)
    
        # x = self.hidden2(x)
        # x = F.leaky_relu_(self.bnhidden2(x), negative_slope=0.01)
        
        
        
        outputs = self.output(x)
        return outputs

In [ ]:
model = neuralNetwork(input_dim, 20,20, output_dim)

In [ ]:
criterion = torch.nn.CrossEntropyLoss(weight=class_weights.float())

In [ ]:
# sgd ya da adam kullanilabilir
# optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)
from torch.optim.lr_scheduler import LambdaLR
from torch.optim.lr_scheduler import MultiStepLR
# optimizer = torch.optim.SGD(model.parameters(), lr = lr_rate, momentum=0.9)
optimizer = torch.optim.SGD(model.parameters(), lr = lr_rate)
# learning rate'in 20 ve 40. epochta 10da 1'ine dusmesini sagladim
# scheduler = MultiStepLR(optimizer, milestones=[240], gamma=0.5)

In [ ]:
import datetime
a = datetime.datetime.now().replace(microsecond=0)
for epoch in range(epochs):
    for i, (features, labels) in enumerate(train_loader):
        features = Variable(features)
        labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(features.float())
        loss = criterion(outputs.float(), labels)
        loss.backward()
        optimizer.step()

        
        if i % (len(X_train) // (3 * batch_size))  == 0:
            # calculate Accuracy
            correct = 0
            total = 0
            for features, labels in val_loader:
                features = Variable(features)
                with torch.no_grad():
                    outputs = model(features.float())
                _, predicted = torch.max(outputs.data, 1)
                total+= labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                correct+= (predicted == labels).sum()
            accuracy = 100 * correct/total
            # if(epoch % 5 == 0 and i == 0):
            #     print("Epoch: {}. Iteration: {}. Loss: {}. Accuracy: {}.".format(epoch, i, loss.item(), accuracy))
            print("Epoch: {}. Iteration: {}. Loss: {}. Accuracy: {}.".format(epoch, i, loss.item(), accuracy))

In [ ]:
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

